# Goal 

**Inputs:**

1. Take an input context text 
2. Take input sentence

**Backend:**

1. Convert input(1) to tokens 
2. wikpedia_topic_mode(tokens) -> likely_topics
3. wikpedia_topic_mode(likely_topics) -> pun_possible_words
4. pun.insert_pun( input(2), pun_possible_words ) -> new sentence

**Output:**

1. Print new sentence



In [1]:
# imports 
import pun
import pickle
from gensim.models.ldamodel import LdaModel as Lda
from gensim import corpora
from gensim.models import doc2vec
from gensim.utils import simple_preprocess

In [2]:
# Load the model

wiki_topicmodel = Lda.load('models/180924_wikipedia_model.stemmed.individually_binned.200.gensim.')

# loading the stemmed_dict

with open('180922_stemmed_dict.p', 'rb') as tounpick:
    stemmed_dict = pickle.load(tounpick)

# Loading the doc2vec

wiki_doc2vec = doc2vec.Doc2Vec.load('models/simple_wiki_chunked_doc2vec')

# loading the doc2vec corpus 

with open('models/simple_wiki_chunked_corpus.p', 'rb') as tounpcik:
    wiki_doc2vec_corpus = pickle.load(tounpcik)

# Dealing with words from topic model

In [3]:
def get_words_from_top_topics(topic_list, model, min_topic_prob=0.1, min_word_prob=0.05):
    """
    First finding all of the words
    """
    
    list_of_words = []
    
    topic_list.sort(key=lambda tup: tup[1], reverse=True)
    
    for topic, topic_prob in topic_list:
        
        if topic_prob < min_topic_prob:
            break
        
        for word_id, word_prob in model.get_topic_terms(topic, 100):
            if word_prob < min_word_prob:
                break
            list_of_words.append(model.id2word[word_id])
    
    return list_of_words

In [4]:
def get_words_from_top_topic(topic_list, model, min_word_prob=0.05):
    """
    First finding all of the words
    """
    
    list_of_words = []
    topic_list.sort(key=lambda tup: tup[1], reverse=True)
    
    topic, topic_prob = topic_list[0]
        
    for word_id, word_prob in model.get_topic_terms(topic, 100):
        if word_prob < min_word_prob:
            break
        if model.id2word[word_id] in stemmed_dict:
            for word in stemmed_dict[model.id2word[word_id]]:
                list_of_words.append(word)
        else:
            list_of_words.append(model.id2word[word_id])

    
    return list_of_words, topic_prob

In [5]:
# Define the context

context = 'purr'
context = "Climbing is the activity of using one's hands, feet, or any other part of the body to ascend a steep object. It is done for locomotion, recreation and competition, in trades that rely on it, and in emergency rescue and military operations. It is done indoors and out, on natural and man-made structures."
# context = "The domestic cat (Felis silvestris catus or Felis catus)[1][4] is a small, typically furry, carnivorous mammal. They are often called house cats[5] when kept as indoor pets or simply cats when there is no need to distinguish them from other felids and felines. They are often valued by humans for companionship and for their ability to hunt vermin. There are more than seventy cat breeds recognized by various cat registries."

# Tokenize context 

context_tokens = pun.tokenize(context)

# Convert to bow

bag_of_words = wiki_topicmodel.id2word.doc2bow(context_tokens)

In [6]:
def sentence_to_topicmodel_words(sentence, model):
    context_tokens = pun.tokenize(sentence)
    bag_of_words = model.id2word.doc2bow(context_tokens)
    document_topics = model.get_document_topics(bag_of_words) 
    return get_words_from_top_topic(document_topics, model)

In [7]:
sentence_to_topicmodel_words(context, wiki_topicmodel)

(['reporter',
  'reporters',
  'news',
  'host',
  'active',
  'activator',
  'activate',
  'activism',
  'activators',
  'actively',
  'activity',
  'activities',
  'activation'],
 0.111388884)

# Dealing with doc2vec information

In [8]:
def sentence_to_doc2vec(text, model):
    """
    Iterator which spits out words that are found in 
    the most 'topical' textual elements
    """
    # parse the sentence 
    text = simple_preprocess(text)
    # Find the respective doc2vec vector
    text_vector = model.infer_vector(text)
    # find the most similar text pieces
    most_similar_documents_with_score = model.docvecs.most_similar([text_vector])
    
    for document_id, cosine_sim_score in most_similar_documents_with_score:
        
        yield (set(pun.tokenize(wiki_doc2vec_corpus[document_id], stem=False, initial_word_split=False)), cosine_sim_score)
        
    

In [9]:
output = sentence_to_doc2vec("Climbing is the activity of using one's hands, feet, or any other part of the body to ascend a steep object. It is done for locomotion, recreation and competition, in trades that rely on it, and in emergency rescue and military operations. It is done indoors and out, on natural and man-made structures.",
                             wiki_doc2vec)

In [10]:
next(output)

/home/wmvoje/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


({'boulder',
  'case',
  'climb',
  'climber',
  'drop',
  'gym',
  'harness',
  'heights',
  'injuries',
  'practice',
  'protect',
  'rock',
  'rope',
  'suffer',
  'type'},
 0.7038991451263428)

# Builiding how it is all connected

In [11]:
def generate_possible_pun_substitutions(context, input_sentence, w2v_number=4):
    """
    Takes context and input sentence 
    
    returns list of possible substitutions with scores and the topic 
    words consdiered
    """
    
    # First process context
    doc2vec_word_generator = sentence_to_doc2vec(context, wiki_doc2vec)
    topic_words, topic_score = sentence_to_topicmodel_words(context, wiki_topicmodel)

    # Then try to generate sentences using these metrics
    output = []
    topic_words_considered = []
    
    # consider word2vec words
    for i in range(w2v_number):
        words, w2v_score = next(doc2vec_word_generator)

        topic_words_considered.extend([[word, 'word2vec', i+1, w2v_score] for word in words])

        sub_tuples = pun.enumerate_PD_pun_subs(input_sentence, words)
        # word, sub_index, phonetic_distance

        for word, sub_index, phon_dist in sub_tuples:
            output.append([word, sub_index, phon_dist, 'word2vec', i+1, w2v_score, phon_dist/w2v_score])

    # Now do topic words
    sub_tuples = pun.enumerate_PD_pun_subs(input_sentence, topic_words)
    for word, sub_index, phon_dist in sub_tuples:

        topic_words_considered.append([word, 'topicModel', 1, topic_score])

        output.append([word, sub_index, phon_dist, 'topicModel', 1, topic_score, phon_dist/topic_score])

    output.sort(key=lambda x: x[6])
    
    return output, topic_words_considered


In [21]:
# context = "Climbing is the activity of using one's hands, feet, or any other part of the body to ascend a steep object. It is done for locomotion, recreation and competition, in trades that rely on it, and in emergency rescue and military operations. It is done indoors and out, on natural and man-made structures."
context = "Prince of the United Kingdom of Great Britain and Northern Ireland is a royal title normally granted to sons and grandsons of reigning and past British monarchs. It is also held by the Duke of Edinburgh, husband and consort of Queen Elizabeth II. The title is granted by the reigning monarch, who is the fount of all honours, through the issuing of letters patent as an expression of the royal will."

input_sentence = "join us at our gym we hope to see you there"

In [22]:
ranked_substitutions, topic_words_considered = generate_possible_pun_substitutions(context, input_sentence)

/home/wmvoje/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [23]:
for thing in [pun.substitute_pun(input_sentence, x[:3]) for x in ranked_substitutions[:10]]:
    print(thing)

join us at flower gym we hope to see you there
join us at our gym we hope to see use there
join us at our gym we hope to see you wear
join us at our gym we hope to sir you there
join us at our gym way hope to see you there
join us at our gym see hope to see you there
join house at our gym we hope to see you there
join us coat our gym we hope to see you there
join us at house gym we hope to see you there
join use at our gym we hope to see you there


In [20]:
ranked_substitutions

[['rope', 6, 2, 'word2vec', 1, 0.6688772439956665, 2.990085277909316],
 ['tube', 7, 2, 'word2vec', 2, 0.6532958149909973, 3.0614002938738567],
 ['tow', 7, 2, 'word2vec', 4, 0.6273124814033508, 3.1882037410220687],
 ['ski', 8, 2, 'word2vec', 4, 0.6273124814033508, 3.1882037410220687],
 ['use', 9, 2, 'word2vec', 4, 0.6273124814033508, 3.1882037410220687],
 ['case', 1, 3, 'word2vec', 1, 0.6688772439956665, 4.485127916863974],
 ['run', 1, 3, 'word2vec', 2, 0.6532958149909973, 4.5921004408107855],
 ['shoot', 2, 3, 'word2vec', 2, 0.6532958149909973, 4.5921004408107855],
 ['swim', 4, 3, 'word2vec', 2, 0.6532958149909973, 4.5921004408107855],
 ['lung', 1, 3, 'word2vec', 3, 0.6473333239555359, 4.634397595458973],
 ['path', 2, 3, 'word2vec', 3, 0.6473333239555359, 4.634397595458973],
 ['matter', 2, 3, 'word2vec', 3, 0.6473333239555359, 4.634397595458973],
 ['stay', 7, 3, 'word2vec', 3, 0.6473333239555359, 4.634397595458973],
 ['race', 1, 3, 'word2vec', 4, 0.6273124814033508, 4.782305611533103],


In [24]:
topic_words_considered

[['coat', 'word2vec', 1, 0.7339661121368408],
 ['arm', 'word2vec', 1, 0.7339661121368408],
 ['issue', 'word2vec', 1, 0.7339661121368408],
 ['marshal', 'word2vec', 1, 0.7339661121368408],
 ['elect', 'word2vec', 1, 0.7339661121368408],
 ['processions', 'word2vec', 1, 0.7339661121368408],
 ['peer', 'word2vec', 1, 0.7339661121368408],
 ['charge', 'word2vec', 1, 0.7339661121368408],
 ['earl', 'word2vec', 1, 0.7339661121368408],
 ['college', 'word2vec', 1, 0.7339661121368408],
 ['give', 'word2vec', 1, 0.7339661121368408],
 ['duke', 'word2vec', 1, 0.7339661121368408],
 ['house', 'word2vec', 1, 0.7339661121368408],
 ['hereditary', 'word2vec', 1, 0.7339661121368408],
 ['member', 'word2vec', 1, 0.7339661121368408],
 ['lord', 'word2vec', 1, 0.7339661121368408],
 ['royal', 'word2vec', 1, 0.7339661121368408],
 ['ceremonies', 'word2vec', 1, 0.7339661121368408],
 ['household', 'word2vec', 1, 0.7339661121368408],
 ['permission', 'word2vec', 1, 0.7339661121368408],
 ['prize', 'word2vec', 2, 0.727767646

In [16]:
list_of_words = [word[0] for word in topic_words_considered]

In [25]:
pun.pos_tag(['choose'])

[('choose', 'NN')]

1. right topic 
2. Does it sound similar 

implement live validation *********

Conversational corpus 

multi-arm banding 


